In [1]:
from utils.binaries import *
from utils.plotting import *
from utils.Auger.FD import PixelPlot, XYScanner

12:16:04 (   +4.5s) [INFO   ] -- import logging
12:16:04 (  +173ms) [INFO   ] -- import numpy as np
12:16:04 (   +11ms) [INFO   ] -- import uncertainties
12:16:04 (  +385ms) [INFO   ] -- import pandas as pd
12:16:04 (    +2ms) [WARNING] -- pandas support to be dropped at some point!
12:16:04 (    +2ms) [INFO   ] -- import binaries.tools as tools
12:16:05 (  +544ms) [INFO   ] -- import matplotlib.pyplot as plt
12:16:06 (  +550ms) [INFO   ] -- import seaborn as so
12:16:06 (    +7ms) [INFO   ] -- set MONI_PATH = PosixPath('/cr/work/filip/monit_and_sd')
12:16:06 (    +1ms) [INFO   ] -- set HIST_PATH = PosixPath('/cr/work/filip/monit_and_sd')
12:16:06 (    +1ms) [INFO   ] -- set PLOT_PATH = PosixPath('/cr/data01/filip/plots')
12:16:06 (    +1ms) [INFO   ] -- set DATA_PATH = PosixPath('/cr/data01/filip/Data')
12:16:06 (    +2ms) [INFO   ] -- import plotting.tools as plot
12:16:06 (   +14ms) [DEBUG  ] -- font size set to 9.5
12:16:06 (    +1ms) [DEBUG  ] -- label size set to 13.0
12:16:06 ( 

In [2]:
import os
from collections import defaultdict

SCANNER_RUNLISTS = "/cr/data01/filip/xy-calibration/config/calib_runlists/"

def reject(run):
    if not run: return True
    # elif run == '16407': return False      # test
    
    else:
        return False

run_data = defaultdict(list)
for LIST in os.listdir(SCANNER_RUNLISTS):
    if 'va' in LIST or 'testing' in LIST: continue

    runlist = XYScanner.load_runlist(LIST[11:18])
    runlist = XYScanner.get_good_runs(runlist)

    for run, info in runlist.iterrows():
        if reject(run): continue

        run_data[info['tel'].upper()].append([run, info['date']])

In [4]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

for tel, runs in run_data.items():

    # if "LL" not in tel: continue
    n_measurements = len(runs)

    fig, axes = plt.subplots(1, n_measurements + 1, figsize=[n_measurements * 3.3, 3.3], width_ratios=[1] * n_measurements + [0.05])
    for ax, (_id, _date) in zip(axes, runs): 
        ax.set_title(f"Run #{_id} on {_date}")

        std, xy = np.loadtxt(f'/cr/data01/filip/xy-calibration/results/outCorr_{_id}.txt', usecols=[3, 4], unpack=True)

        if tel == "LL1":
            pixel_mask = np.loadtxt('/cr/data01/filip/xy-calibration/config/pixel_masks/ll1.txt', usecols=[1], dtype=bool)
            xy[~pixel_mask] = np.nan
            std[~pixel_mask] = np.nan
        elif tel == "LL6":
            pixel_mask = np.loadtxt('/cr/data01/filip/xy-calibration/config/pixel_masks/ll6-xp3062.txt', usecols=[1], dtype=bool)
            xy[~pixel_mask] = np.nan
            std[~pixel_mask] = np.nan

        PixelPlot(ratio := xy / std, ax=ax, cmap='Reds', vmin=1, vmax=2)

        ax.text(0.5, 0.96, fr"$\tilde{{r}}$ = {np.nanmedian(ratio):.2f}, $\breve{{r}} = {np.nanmin(ratio):.2f}$, $\hat{{r}} = {np.nanmax(ratio):.2f}$",
                 ha='center',
                 va='top',
                 transform=ax.transAxes,
                 bbox=dict(boxstyle="round",ec='none',fc='white',alpha=0.7))

    plt.subplots_adjust(wspace=0.04)
    axes[0].text(-0.07, 0.5, tel,
                 ha='center',
                 va='center',
                 rotation=90,
                 transform=axes[0].transAxes,
                 fontsize='xx-large')
    plt.subplots_adjust(wspace=0.04)
    plt.colorbar(ScalarMappable(cmap='Reds', norm=Normalize(1, 2)), cax=axes[-1], shrink=0.7, label="XY calib. corr. / std. calib.")

    plot.save(fig, f'calXY_vs_calA_pixel_plots/{tel}')
    plt.close(fig)
    # break


/tmp/ipykernel_1928562/417004239.py:24: RuntimeWarning: divide by zero encountered in divide
  PixelPlot(ratio := xy / std, ax=ax, cmap='Reds', vmin=1, vmax=2)
